We'll be working to clean and analyze a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. I'm using a sampled set from the original set found on Kaggle (https://www.kaggle.com/orgesleka/used-cars-database/data).

In [68]:
#importing modules
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding="Latin-1")

In [69]:
#return columns and data types
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

In [70]:
#preview the first few rows of the data
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The data has 19 columns for each car. I'm noticing a few things:

1. I'm noticing some of the columns look like they're written in German, which we may need to take into account. 
2. I'm also noticing some of the columns I would expect to be datetime / int / float are objects, for example dateCrawled is an object (should be datetime), price is an object (should be float or int), and odometer is an object (should be float or int). So I'm anticipating that I'll clean these columns to make the data more easy with which to work. 
3. Some columns, like vehiclyType, have some null values. But it's only 5 columns that have null values, and they have less than 20% null values, so not actually that many out of the 19 and it's not significant in the columns that do have null values. I'm guessing I'll investigate those.

In [71]:
#clean the column names
columns = list(autos.columns)
columns[7] = "registration_year"
columns[12] = "registration_month"
columns[15] = "unrepaired_damage"
columns[16] = "ad_created"

def change_case(str): 
    res = [str[0].lower()] 
    for c in str[1:]: 
        if c in ('ABCDEFGHIJKLMNOPQRSTUVWXYZ'): 
            res.append('_') 
            res.append(c.lower()) 
        else: 
            res.append(c) 
      
    return ''.join(res)

index = 0
for i in columns:
    columns[index] = change_case(i)
    index += 1

autos.columns = columns
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [72]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-11 22:38:16,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


1. Columns to drop include seller and offer_type which have "privat" and "Angebot" 49,999 times.
2. Columns that need more investigation include vehicle_type, gearbox, model, fuel_type, and unrepaired_damage because they have null values. 
3. Columns to change data types for include date_crawled, price, regisration_year, power_p_s, odometer, and ad_created.


In [73]:
#convert price to numeric data type
autos['price'] = autos['price'].str.replace('$','')
autos['price'] = autos['price'].str.replace(',','')
autos['price'] = pd.to_numeric(autos['price'])

In [74]:
#convert odometer to numeric data type
autos['odometer'] = autos['odometer'].str.replace('k','')
autos['odometer'] = autos['odometer'].str.replace('m','')
autos['odometer'] = autos['odometer'].str.replace(',','')
autos['odometer'] = pd.to_numeric(autos['odometer'])

In [75]:
autos["odometer"]

0        150000
1        150000
2         70000
3         70000
4        150000
          ...  
49995    100000
49996    150000
49997      5000
49998     40000
49999    150000
Name: odometer, Length: 50000, dtype: int64

In [76]:
autos.rename(columns={"odometer": "odometer_km"}, inplace=True)

In [77]:
autos["price"].unique().shape
#There are 2,357 unqiue values

(2357,)

In [78]:
autos["price"].describe()
#mean price is $9,840
#50% of all data falls within range of $1,100 to $7,200

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [79]:
autos["price"].value_counts().sort_index(ascending=True)
#looks like we do have some outliers, although they're all in the realm of possibility except for cars listed for $0 or close to $0 and a car with a price of $99,999,999. Seem like data entry errors to me. Let's explore these entries further.

0           1421
1            156
2              3
3              1
5              2
            ... 
10000000       1
11111111       2
12345678       3
27322222       1
99999999       1
Name: price, Length: 2357, dtype: int64

In [80]:
autos[(autos["price"]>10000000)]
#Upon google searching, a Golf Schlachte and Fiat Punto are definitely not worth in the 10s of millions of dollars. That makes me think these entries, and likely additional entries with even lower prices, are outliers I should remove.

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
2897,2016-03-12 21:50:57,Escort_MK_1_Hundeknochen_zum_umbauen_auf_RS_2000,privat,Angebot,11111111,test,limousine,1973,manuell,48,escort,50000,3,benzin,ford,nein,2016-03-12 00:00:00,0,94469,2016-03-12 22:45:27
24384,2016-03-21 13:57:51,Schlachte_Golf_3_gt_tdi,privat,Angebot,11111111,test,NaN,1995,NaN,0,NaN,150000,0,NaN,volkswagen,NaN,2016-03-21 00:00:00,0,18519,2016-03-21 14:40:18
27371,2016-03-09 15:45:47,Fiat_Punto,privat,Angebot,12345678,control,NaN,2017,NaN,95,punto,150000,0,NaN,fiat,NaN,2016-03-09 00:00:00,0,96110,2016-03-09 15:45:47
39377,2016-03-08 23:53:51,Tausche_volvo_v40_gegen_van,privat,Angebot,12345678,control,NaN,2018,manuell,95,v40,150000,6,NaN,volvo,nein,2016-03-08 00:00:00,0,14542,2016-04-06 23:17:31
39705,2016-03-22 14:58:27,Tausch_gegen_gleichwertiges,privat,Angebot,99999999,control,limousine,1999,automatik,224,s_klasse,150000,9,benzin,mercedes_benz,NaN,2016-03-22 00:00:00,0,73525,2016-04-06 05:15:30
42221,2016-03-08 20:39:05,Leasinguebernahme,privat,Angebot,27322222,control,limousine,2014,manuell,163,c4,40000,2,diesel,citroen,NaN,2016-03-08 00:00:00,0,76532,2016-03-08 20:39:05
47598,2016-03-31 18:56:54,Opel_Vectra_B_1_6i_16V_Facelift_Tuning_Showcar...,privat,Angebot,12345678,control,limousine,2001,manuell,101,vectra,150000,3,benzin,opel,nein,2016-03-31 00:00:00,0,4356,2016-03-31 18:56:54


In [81]:
autos[(autos["price"]<10)]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
27,2016-03-27 18:45:01,Hat_einer_Ahnung_mit_Ford_Galaxy_HILFE,privat,Angebot,0,control,NaN,2005,NaN,0,NaN,150000,0,NaN,ford,NaN,2016-03-27 00:00:00,0,66701,2016-03-27 18:45:01
55,2016-03-07 02:47:54,Mercedes_E320_AMG_zu_Tauschen!,privat,Angebot,1,test,NaN,2017,automatik,224,e_klasse,125000,7,benzin,mercedes_benz,nein,2016-03-06 00:00:00,0,22111,2016-03-08 05:45:44
71,2016-03-28 19:39:35,Suche_Opel_Astra_F__Corsa_oder_Kadett_E_mit_Re...,privat,Angebot,0,control,NaN,1990,manuell,0,NaN,5000,0,benzin,opel,NaN,2016-03-28 00:00:00,0,4552,2016-04-07 01:45:48
80,2016-03-09 15:57:57,Nissan_Primera_Hatchback_1_6_16v_73_Kw___99Ps_...,privat,Angebot,0,control,coupe,1999,manuell,99,primera,150000,3,benzin,nissan,ja,2016-03-09 00:00:00,0,66903,2016-03-09 16:43:50
87,2016-03-29 23:37:22,Bmw_520_e39_zum_ausschlachten,privat,Angebot,0,control,NaN,2000,NaN,0,5er,150000,0,NaN,bmw,NaN,2016-03-29 00:00:00,0,82256,2016-04-06 21:18:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49884,2016-03-11 13:55:30,Audi_a6_2.5l__Schnaeppchen_nur_heute,privat,Angebot,0,test,kombi,1999,manuell,150,a6,150000,11,diesel,audi,NaN,2016-03-11 00:00:00,0,27711,2016-03-12 03:17:08
49943,2016-03-16 20:46:08,Opel_astra,privat,Angebot,0,control,NaN,2016,manuell,101,astra,150000,8,benzin,opel,NaN,2016-03-16 00:00:00,0,89134,2016-03-17 19:44:20
49960,2016-03-25 22:51:55,Ford_KA_zu_verschenken_***Reserviert***,privat,Angebot,0,control,kleinwagen,1999,manuell,60,ka,150000,6,benzin,ford,NaN,2016-03-25 00:00:00,0,34355,2016-03-25 22:51:55
49974,2016-03-20 10:52:31,Golf_1_Cabrio_Tuev_Neu_viele_Extras_alles_eing...,privat,Angebot,0,control,cabrio,1983,manuell,70,golf,150000,2,benzin,volkswagen,nein,2016-03-20 00:00:00,0,8209,2016-03-27 19:48:16


In [82]:
#So I need to look at the data more granularly to decide where my outlier cutoff points will be.
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

In [83]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

There are about 1400 entries with a price of \$0 I will keep any rows with a price above \$0 because this is eBay, a \$1 price may be legitimate. There are also prices that steadily rise until \$350,000, when the price jumps to \$999,990. I will keep any rows with a price up to \$350k.

In [84]:
autos = autos[autos["price"].between(1,351000)]
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

In [103]:
autos["odometer_km"].shape
#There are 48,565 unqiue values

(48545,)

In [104]:
autos["odometer_km"].describe()
#min is 5k miles, max is 150k miles. these all seem legitimate.

count     48545.000000
mean     125819.136883
std       39723.379171
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [105]:
autos["odometer_km"].value_counts()
#So maybe this is a preset field the seller can pick from. The values look rounded, with significant gaps between values. So will not eliminate any of these rows based on outlier data.

150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
70000      1217
60000      1155
50000      1012
5000        823
40000       815
30000       780
20000       762
10000       246
Name: odometer_km, dtype: int64

In [88]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [89]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

Looks like thsi data was crawled over a range of 34 days, from 3/5/2016 to 4/7/2016. Highest percentage of posts is on 4/3/2016, with 3.9% of the total posts, although this isn't an outlier. Many of the days have 3 - 4% of the total posts.

In [90]:
pd.set_option('display.max_rows', 100)
(autos['ad_created']
    .str[:10]
    .value_counts(normalize=True, dropna=False)
    .sort_index()
    )

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
2016-02-22    0.000021
2016-02-23    0.000082
2016-02-24    0.000041
2016-02-25    0.000062
2016-02-26    0.000041
2016-02-27    0.000124
2016-02-28    0.000206
2016-02-29    0.000165
2016-03-01    0.000103
2016-03-02    0.000103
2016-03-03    0.000865
2016-03-04    0.001483
2016-03-05    0.022897
2016-03-06 

This field is from the website, looks like a 76 day range of values (non-contiguous) from 6/11/2015 to 4/7/2016. The percentage is less than 1% until 3/4/2016, when it begins to steadily climb until 3.5% on 3/7/2016, fluctuating from 3.9% to close to 0% for the remaining days until 4/7/2016.

In [91]:
autos['last_seen'].str[:10].describe()

count          48565
unique            34
top       2016-04-06
freq           10772
Name: last_seen, dtype: object

In [92]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

Same date range of 34 days for date_crawled, from 3/5/2016 to 4/7/2016. The last 3 days see much higher percentages of posts last seen than the other days. Maybe this is due to how the crawler counts the last time it saw a post before ending? Or maybe there was a big spike in the selling of cars on those dates for some reason, although that seems unlikely.

In [93]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Looks like some of this registration year data is inaccurate. A car could not be registered in the year 1000 or 9999. Let's look at these cases more in depth and brainstorm what we can do about them.

In [94]:
autos["registration_year"].value_counts().sort_index(ascending=True).head(20)

1000    1
1001    1
1111    1
1800    2
1910    5
1927    1
1929    1
1931    1
1934    2
1937    4
1938    1
1939    1
1941    2
1943    1
1948    1
1950    3
1951    2
1952    1
1953    1
1954    2
Name: registration_year, dtype: int64

I am going to remove all years before 1910. Based on some general internet research, seems the first car was invented by Carl Benz in 1885/1886.

In [95]:
autos["registration_year"].value_counts().sort_index(ascending=False).head(20)

9999       3
9000       1
8888       1
6200       1
5911       1
5000       4
4800       1
4500       1
4100       1
2800       1
2019       2
2018     470
2017    1392
2016    1220
2015     392
2014     663
2013     803
2012    1310
2011    1623
2010    1589
Name: registration_year, dtype: int64

I'll keep any year until 2019, as 2800 is not a valid year.

In [96]:
autos = autos[autos["registration_year"].between(1910,2020)]
autos["registration_year"].describe()

count    48545.000000
mean      2003.461510
std          7.566196
min       1910.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       2019.000000
Name: registration_year, dtype: float64

In [97]:
autos["registration_year"].value_counts(normalize=True)

2000    0.065012
2005    0.060480
1999    0.059677
2004    0.055680
2003    0.055598
2006    0.055001
2001    0.054300
2002    0.051210
1998    0.048676
2007    0.046905
2008    0.045628
2009    0.042950
1997    0.040190
2011    0.033433
2010    0.032733
2017    0.028674
1996    0.028283
2012    0.026985
1995    0.025276
2016    0.025131
2013    0.016541
2014    0.013657
1994    0.012957
2018    0.009682
1993    0.008755
2015    0.008075
1992    0.007622
1990    0.007148
1991    0.006983
1989    0.003584
1988    0.002781
1985    0.001957
1980    0.001751
1987    0.001483
1986    0.001483
1983    0.001051
1984    0.001051
1978    0.000906
1982    0.000845
1970    0.000783
1979    0.000700
1972    0.000680
1981    0.000577
1968    0.000536
1971    0.000536
1967    0.000536
1974    0.000494
1960    0.000474
1973    0.000474
1977    0.000453
1966    0.000453
1976    0.000433
1969    0.000391
1975    0.000371
1965    0.000350
1964    0.000247
1963    0.000165
1959    0.000124
1961    0.0001

Looks like most cars being sold were registered in the late 1990s or early 2000s.

In [98]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.212813
opel              0.108641
bmw               0.108641
mercedes_benz     0.095787
audi              0.085858
ford              0.069667
renault           0.047894
peugeot           0.029457
fiat              0.025996
seat              0.018931
skoda             0.016068
nissan            0.015264
mazda             0.015223
smart             0.014296
citroen           0.014111
toyota            0.012586
hyundai           0.009950
sonstige_autos    0.009641
volvo             0.009043
mini              0.008611
mitsubishi        0.008178
honda             0.007993
kia               0.007107
alfa_romeo        0.006612
porsche           0.005912
suzuki            0.005891
chevrolet         0.005665
chrysler          0.003481
dacia             0.002657
daihatsu          0.002513
jeep              0.002204
subaru            0.002101
land_rover        0.002039
saab              0.001627
daewoo            0.001566
jaguar            0.001524
trabant           0.001380
r

German manaufacturers represent the top 5 brands, at around 50% of thd total overal listings. There are many brands without a significant number of listings (from Hyuandai down, it's less than 1% of listings) so I will remove brands with less than 1% of total listings.

In [109]:
brand_counts = autos['brand'].value_counts(normalize=True)
brands_greater_than_1_percent = brand_counts[brand_counts > 0.01].index
print(brands_greater_than_1_percent)

Index(['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota'],
      dtype='object')


In [122]:
brand_mean_prices = {}

for brand in brands_greater_than_1_percent:
    brand_only = autos[autos['brand'] == brand]
    mean_price = brand_only['price'].mean()
    brand_mean_prices[brand] = mean_price
    
brand_mean_prices

{'volkswagen': 5333.1962055948115,
 'opel': 2941.4664391353813,
 'bmw': 8261.382442169132,
 'mercedes_benz': 8526.623225806452,
 'audi': 9212.9306621881,
 'ford': 3728.4121821407452,
 'renault': 2431.195698924731,
 'peugeot': 3065.611888111888,
 'fiat': 2793.8700475435817,
 'seat': 4320.168661588684,
 'skoda': 6353.544871794872,
 'nissan': 4669.3859649122805,
 'mazda': 4059.059539918809,
 'smart': 3518.102305475504,
 'citroen': 3756.07299270073,
 'toyota': 5148.0032733224225}

- BMW, Mercedes, and Audi are the most expensive brands, all above \$8,200. 
- Then there is a distinct drop off to less expensiove brands like Volkswagen, Seat, Skoda, and Toyota in the \$5,000 to \$6,400 range. Thes brands fall inbetween the less and more expensive brands.
- Then all the remaining brands are less expensive.

In [125]:
brand_mean_mileage = {}

for brand in brands_greater_than_1_percent:
    brand_only = autos[autos['brand'] == brand]
    mean_mileage = brand_only['odometer_km'].mean()
    brand_mean_mileage[brand] = int(mean_mileage)
    
brand_mean_mileage

{'volkswagen': 128955,
 'opel': 129452,
 'bmw': 132682,
 'mercedes_benz': 130848,
 'audi': 129492,
 'ford': 124349,
 'renault': 128062,
 'peugeot': 127356,
 'fiat': 117567,
 'seat': 121768,
 'skoda': 110993,
 'nissan': 118711,
 'mazda': 124871,
 'smart': 100511,
 'citroen': 120160,
 'toyota': 116219}

In [126]:
brand_mean_prices_series = pd.Series(brand_mean_prices)
brand_mean_mileage_series = pd.Series(brand_mean_mileage)

volkswagen       5333.196206
opel             2941.466439
bmw              8261.382442
mercedes_benz    8526.623226
audi             9212.930662
ford             3728.412182
renault          2431.195699
peugeot          3065.611888
fiat             2793.870048
seat             4320.168662
skoda            6353.544872
nissan           4669.385965
mazda            4059.059540
smart            3518.102305
citroen          3756.072993
toyota           5148.003273
dtype: float64

In [133]:
brand_dataframe = pd.DataFrame(brand_mean_prices_series, columns = ['mean_price'])
brand_dataframe["mean_mileage"] = brand_mean_mileage_series
brand_dataframe

,mean_price,mean_mileage
volkswagen,5333.196206,128955
opel,2941.466439,129452
bmw,8261.382442,132682
mercedes_benz,8526.623226,130848
audi,9212.930662,129492
ford,3728.412182,124349
renault,2431.195699,128062
peugeot,3065.611888,127356
fiat,2793.870048,117567
seat,4320.168662,121768


It appears the more expensive brands are associated with higher mileage, as BMW, Mercedes, and Audi are above 129k miles, which is close to BMW's max of 133k miles. The less expensive cars, like smart, fiat, and ford are associated with lower mileage, although opel looks like an outlier in this relationship (low price, high mileage).